<a href="https://colab.research.google.com/github/adventHymnals/resources/blob/master/Update_VegM_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Website Tools

Run this noteboook to update(push changes) to website once you have edited [the website config sheet](https://docs.google.com/spreadsheets/d/1-otVkXNn6i4tWGki9grXzc4DjG6TdQ_XWTBbJttlESU/edit#gid=512290428).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install gdown

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%bash
dirPath=""
if [ -d "/content/drive/MyDrive/Publishing/Website" ]
then
    dirPath="/content/drive/MyDrive/Publishing/Website"
else
    if [ -d "/content/drive/My Drive/Publishing/Website" ]
    then
        dirPath="/content/drive/My Drive/Publishing/Website"
    else
        exit
    fi
fi

#### ssh key
mkdir -p ~/.ssh
cp "$dirPath/private/id_rsa" ~/.ssh/

#### download google sheet
docId="1-otVkXNn6i4tWGki9grXzc4DjG6TdQ_XWTBbJttlESU"  ## Hardcode this (for now)
sheetId="1050384173"                                  ## Hardcode this (for now)
## hardcode also folderId,,, let these be the first in the list....
rm -f logs1
wget --output-file="logs1" "https://docs.google.com/spreadsheets/d/$docId/export?format=csv&gid=$sheetId" -O "sheet.csv"
rm logs1

#### process sheet vars
# prelink=$(grep -rh "Github," sheet.csv | sed 's/Github,//')
#prelink=$(cat sheet.csv |grep -o 'Github,[^ ]*' | cut -d',' -f2 | tr -d '\r')
#prelink=$(cat sheet.csv | awk -F',' '/Github/ {print $2}') | tr -d '\r'
prelink=$(cat sheet.csv |grep -o 'Github,[^ ]*' | cut -d',' -f2 | tr -d '\r')
link="git@github.com:$prelink.git"

GitCommitUser=$(grep -rh "GitCommitUser," sheet.csv | sed 's/GitCommitUser,//' | tr -d '\r')
GitCommitEmail=$(grep -rh "GitCommitEmail," sheet.csv | sed 's/GitCommitEmail,//'| tr -d '\r')
rm sheet.csv


#### clone repo
rm -rf websiteRepo
mkdir -p websiteRepo

ssh-keyscan GitHub.com >> /root/.ssh/known_hosts 2>&1 >/dev/null
chmod 644 /root/.ssh/known_hosts
chmod 600 /root/.ssh/id_rsa
git clone "$link" websiteRepo 


#### Download sheets
cd websiteRepo
mkdir -p "./configs/sheets"
rm -f logs
while IFS=, read -r docId sheetId; do
    wget --output-file="logs" "https://docs.google.com/spreadsheets/d/$docId/export?format=csv&gid=$sheetId" -O "./configs/sheets/$docId$sheetId.csv"
done < "./configs/datasheets.txt"
rm -f logs

## merge images
rsync -aq  /content/drive/MyDrive/Publishing/Website/img/ ./assets/img/


## Install pandoc to convert _posts to markdown
## To use pandoc, we will first need to convert the gdoc files to docx. This we reserve for later
#curl --silent "https://api.github.com/repos/jgm/pandoc/releases/latest"|   grep "browser_download_url.*amd64.deb" | head -n 1 | cut -d : -f 2,3 | tr -d \"  | xargs wget -O tmp.deb && sudo dpkg -i tmp.deb
rm -rf /tmp/_posts
mkdir -p /tmp/_posts
#ls -lha /content/drive/MyDrive/Publishing/Website/_posts
####### ls -lha /content/drive/MyDrive/Publishing/Website/_posts/
#### #### #### rsync -avz --ignore-errors  /content/drive/MyDrive/Publishing/Website/_posts/ /tmp/_posts/
# gdown --id 17yR0j6LMb6hG1E8DRz0tslvxCk4o5vMn -O /tmp/_posts
folderId="17yR0j6LMb6hG1E8DRz0tslvxCk4o5vMn"

## this will fail for .gdown files. Just use it to get the docIds
gdown https://drive.google.com/drive/folders/17yR0j6LMb6hG1E8DRz0tslvxCk4o5vMn -O /tmp --folder --continue > /tmp/filelist 2>/dev/null || true
currentDir=$(pwd)
cd /tmp/_posts
while read -r docId; do
  echo "Exporting $docId"
  # Download a served file keeping its name.
  # wget --content-disposition --trust-server-names seems to be having some errors

  # wget --content-disposition --trust-server-names "https://docs.google.com/document/d/${docId}/export?format=docx" || true
  curl -sJLO "https://docs.google.com/document/d/${docId}/export?format=docx" || true
done < <(grep -o 'Processing file .*' /tmp/filelist |sed 's/Processing file //' | sed 's/ .*//') 

find . -name '*export?format=docx*' -type f -delete

## download conversion script
wget -q https://raw.githubusercontent.com/adventHymnals/resources/master/scripts/docx2md.sh -O docx2md.sh
chmod +x docx2md.sh
for file in /tmp/_posts/*.docx; do
    filename=$(basename "$file" .docx)
    ./docx2md.sh "$filename"
done

cd "$currentDir"

frontMatter=""
mkdir -p ./assets/img/posts/
for file in ./_posts/*; do
  echo $file
  filename=$(basename "$file")
  echo $filename
  pattern="^[0-9]{4}-[0-9]{2}-[0-9]{2}-(.*)\.(md|markdown)$"
  if [[ "${filename}" =~ ${pattern} ]]; then
      post_title="${BASH_REMATCH[1]}"
      echo "${post_title}"
      if [[ -d "/tmp/_posts/$post_title" ]]; then
          filename1=$(basename $file)
            
          # remove the extension (either .md or .markdown)
          filename1="${filename1%.*}"

          sed -i "s|./media|{{site.baseurl}}assets/img/posts/$filename1/media|g" "/tmp/_posts/$post_title/README.md" # correct links
          sed -i 's/\\\(["`\-]\)/\1/g' "/tmp/_posts/$post_title/README.md"
          mv "/tmp/_posts/$post_title/README.md" $file
          ## copy media as well
          ## edit links in .md for media
          ## check if dir is not empty (has media)
          if test -n "$(find /tmp/_posts/$post_title -maxdepth 1 -type f -o -type d)"; then
            rsync -aq /tmp/_posts/$post_title/ ./assets/img/posts/$filename1/
            # mv /tmp/_posts/$post_title ./assets/img/
            rm -rf /tmp/_posts/$post_title
          # else
            # 
          fi
      else
          rm -rf ./assets/img/$post_title
          rm -rf $file
      fi
  fi
done

## Handle new posts
for dir in /tmp/_posts/*/; do
    # Do something with each subdirectory
    echo "Processing $dir"
    newPost=$(basename $dir)
    date=$(date +%Y-%m-%d)
    newname="${date}-${newPost}"
    sed -i "s|./media|{{site.baseurl}}assets/img/posts/$newname/media|g" "/tmp/_posts/$newPost/README.md" # correct links
    sed -i 's/\\\(["`\-]\)/\1/g' "/tmp/_posts/$newPost/README.md"
    mv "/tmp/_posts/$newPost/README.md" "./_posts/$newname.md"
    if test -n "$(find /tmp/_posts/$newPost -maxdepth 1 -type f -o -type d)"; then
      # mv /tmp/_posts/$newPost ./assets/img/
      rsync -aq /tmp/_posts/$newPost/ ./assets/img/posts/$newname/
      rm -rf /tmp/_posts/$newPost
    # else
    #   echo "/path/to/directory is empty"
    fi
done


#### push repo
git config --global user.email "$GitCommitEmail" 
git config --global user.name "$GitCommitUser"
git add . && git commit -m "$(date)" && git push origin master



Exporting 1_wnUwdPvtPzIB4ZWNwSvCRFuRBUd5tMgXejcA5Z82Ng
Exporting 1rUVW2gY1OK8SSNc-uRPZUNz5z2_Z8zacuKWEiKzzutM
Exporting 18aKvKgP7gqLKAEykY5TNUz0FO5zNPqkP4l2ewVDakNw
Exporting 1lQaYNXrFFqNc9qkawpete8untEf54gTqtAXfUhAHzlQ
./_posts/2023-04-14-welcome-to-jekyll.markdown
2023-04-14-welcome-to-jekyll.markdown
welcome-to-jekyll
./_posts/2023-04-30-renamed_post.md
2023-04-30-renamed_post.md
renamed_post
./_posts/2023-04-30-Untitleddocument.md
2023-04-30-Untitleddocument.md
Untitleddocument
Processing /tmp/_posts/Personality-of-God/
[master d581f0a] Sun 30 Apr 2023 03:14:28 PM UTC
 1 file changed, 14 insertions(+)
 create mode 100644 _posts/2023-04-30-Personality-of-God.md


Cloning into 'websiteRepo'...
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.        
To github.com:vegministry/website-private.git
   e114c23..d581f0a  master -> master
